# Naive Bayes(Poem Classifier)

自行收集的詩詞資料集

- https://drive.google.com/open?id=1KuH3QyTaD7yrqGv8uTPPVgDQD_XW8amC

- poem_train.csv: 供你訓練模型
- poem_test.csv: 供你測試模型

In [1]:
import pandas as pd
# 為了顯示的漂亮, 我刻意的把印出來的 row 只顯示 15 個和 column 只顯示十個
# 大家練習的時候可以去掉下面兩行
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 10)
# scikit-learn 會有些 deprecation warning, 為了顯示漂亮, 我刻意地忽略掉
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv("poem_train.csv", encoding = "utf-8")
df

,作者,詩名,內容
0,李白,菩薩蠻·平林漠漠煙如織,平林漠漠煙如織，寒山一帶傷心碧。\r\n暝色入高樓，有人樓上愁。玉階空佇立，宿鳥歸飛急。\r...
1,李白,把酒問月,青天有月來幾時，我今停杯一問之：人攀明月不可得，月行卻與人相隨？皎如飛鏡臨丹闕，綠煙滅儘清輝...
2,李白,春思,燕草如碧絲，秦桑低綠枝。當君懷歸日，是妾斷腸時。春風不相識，何事入羅幃。
3,李白,春夜洛城聞笛,誰家玉笛暗飛聲，散入春風滿洛城。此夜曲中聞折柳，何人不起故園情。
4,李白,古風 其十九,西上蓮花山，迢迢見明星。(西上 一作：西嶽)素手把芙蓉，虛步躡太清。霓裳曳廣帶，飄拂升天行。...
5,李白,關山月,明月出天山，蒼茫雲海間。長風幾萬裡，吹度玉門關。漢下白登道，胡窺青海灣。由來征戰地，不見有人...
6,李白,將進酒·君不見黃河之水天上來,君不見黃河之水天上來，奔流到海不複回。君不見高堂明鏡悲白發，朝如青絲暮成雪。人生得意須儘歡，...
...,...,...,...
2724,白居易,秋蝶,秋花紫蒙蒙，秋蝶黃茸茸。花低蝶新小，飛戲叢西東。日暮涼風來，紛紛花落叢。夜深白露冷，蝶已死叢...
2725,白居易,三年為刺史二首,三年為刺史，無政在人口。唯向郡城中，題詩十餘首。慚非甘棠詠，豈有思人不？三年為刺史，飲冰複食...


In [2]:
df['作者'] = df['作者'].astype('category')
saved_map = { cat:df['作者'].cat.categories.get_loc(cat) for cat in df['作者'].cat.categories }
saved_map

{'李白': 0, '杜甫': 1, '白居易': 2}

In [3]:
# 定義好等等我們要對所有內容做的 split
def split_poem(poem):
    return " ".join(jieba.cut(poem))

def process_poems(df):
    df['內容'] = df['內容'].apply(split_poem)
    df['內容'] = df['內容'].str.replace('\r', '')
    df['內容'] = df['內容'].str.replace('\n', '')
    # 詩名我們今天沒用到, 先 drop 掉
    df = df.drop(["詩名"], axis = 1)
    df['作者'] = df['作者'].astype('category')
    return df

In [4]:
import jieba
df = process_poems(df)
df

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.590 seconds.
Prefix dict has been built succesfully.


,作者,內容
0,李白,平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 暝 色入 高樓 ， 有人 樓上 愁 。 ...
1,李白,青天 有 月 來 幾時 ， 我今 停杯 一問 之 ： 人攀 明月 不可 得 ， 月行 卻 與...
2,李白,燕草 如碧絲 ， 秦桑低 綠枝 。 當君 懷歸日 ， 是 妾 斷腸時 。 春風 不 相識 ，...
3,李白,誰 家玉笛 暗飛聲 ， 散 入春 風滿 洛城 。 此 夜曲 中聞折 柳 ， 何人 不起 故園情 。
4,李白,西上 蓮 花山 ， 迢迢 見 明星 。 ( 西 上 一作 ： 西 嶽 ) 素 手把 芙蓉...
5,李白,明月 出 天山 ， 蒼茫 雲海間 。 長 風 幾萬裡 ， 吹度 玉門關 。 漢下 白登道 ，...
6,李白,君不見 黃 河之水 天上 來 ， 奔流 到海 不 複 回 。 君不見 高堂 明鏡 悲白發 ，...
...,...,...
2724,白居易,秋花 紫 蒙蒙 ， 秋蝶 黃 茸茸 。 花低 蝶 新 小 ， 飛戲 叢西東 。 日暮 涼風來...
2725,白居易,三年 為 刺史 ， 無政 在 人口 。 唯向 郡 城中 ， 題 詩十餘首 。 慚非 甘棠 詠...


In [5]:
df['作者'] = df['作者'].cat.codes
df

,作者,內容
0,0,平林 漠漠 煙如織 ， 寒山 一帶 傷心 碧 。 暝 色入 高樓 ， 有人 樓上 愁 。 ...
1,0,青天 有 月 來 幾時 ， 我今 停杯 一問 之 ： 人攀 明月 不可 得 ， 月行 卻 與...
2,0,燕草 如碧絲 ， 秦桑低 綠枝 。 當君 懷歸日 ， 是 妾 斷腸時 。 春風 不 相識 ，...
3,0,誰 家玉笛 暗飛聲 ， 散 入春 風滿 洛城 。 此 夜曲 中聞折 柳 ， 何人 不起 故園情 。
4,0,西上 蓮 花山 ， 迢迢 見 明星 。 ( 西 上 一作 ： 西 嶽 ) 素 手把 芙蓉...
5,0,明月 出 天山 ， 蒼茫 雲海間 。 長 風 幾萬裡 ， 吹度 玉門關 。 漢下 白登道 ，...
6,0,君不見 黃 河之水 天上 來 ， 奔流 到海 不 複 回 。 君不見 高堂 明鏡 悲白發 ，...
...,...,...
2724,2,秋花 紫 蒙蒙 ， 秋蝶 黃 茸茸 。 花低 蝶 新 小 ， 飛戲 叢西東 。 日暮 涼風來...
2725,2,三年 為 刺史 ， 無政 在 人口 。 唯向 郡 城中 ， 題 詩十餘首 。 慚非 甘棠 詠...


In [6]:
test_df = pd.read_csv("poem_test.csv")
test_df = process_poems(test_df)
test_df

,作者,內容
0,李白,日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1,李白,朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2,李白,李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3,李白,故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4,李白,危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。
5,李白,床前 明月光 ， 疑是 地上 霜 。 舉頭 望明月 ， 低頭思 故鄉 。
6,李白,天門 中斷 楚江 開 ， 碧水 東流 至此 回 。 兩岸 青山 相對 出 ， 孤帆 一片 日...
...,...,...
23,白居易,雨 砌 長 寒蕪 ， 風庭 落秋果 。 窗間 有 閒叟 ， 儘 日 看 書 坐 。 書中見 ...
24,白居易,睡足 肢體 暢 ， 晨起 開 中堂 。 初旭泛 簾幕 ， 微風 拂 衣裳 。 二婢 扶 盥櫛...


In [7]:
test_df['作者'] = test_df['作者'].replace(saved_map)
test_df

,作者,內容
0,0,日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1,0,朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2,0,李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3,0,故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4,0,危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。
5,0,床前 明月光 ， 疑是 地上 霜 。 舉頭 望明月 ， 低頭思 故鄉 。
6,0,天門 中斷 楚江 開 ， 碧水 東流 至此 回 。 兩岸 青山 相對 出 ， 孤帆 一片 日...
...,...,...
23,2,雨 砌 長 寒蕪 ， 風庭 落秋果 。 窗間 有 閒叟 ， 儘 日 看 書 坐 。 書中見 ...
24,2,睡足 肢體 暢 ， 晨起 開 中堂 。 初旭泛 簾幕 ， 微風 拂 衣裳 。 二婢 扶 盥櫛...


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
vec = TfidfVectorizer()
bag = vec.fit_transform(df['內容'])
print("維度:", len(vec.get_feature_names()))
clf = MultinomialNB(alpha = 0.001)
clf.fit(bag, df['作者'])

維度: 52294


MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [9]:
from sklearn.metrics import accuracy_score
test_bag = vec.transform(test_df['內容'])
predict = clf.predict(test_bag)
print("預測　　:", list(predict))
print("正確標籤:", list(test_df['作者']))
print("Naive-Bayes 正確率: ", accuracy_score(test_df['作者'], predict) * 100, "%")

預測　　: [0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Naive-Bayes 正確率:  80.0 %


In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_df['作者'], predict)
pd.DataFrame(cm)

,0,1,2
0,8,1,1
1,0,8,2
2,1,1,8


In [11]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=8)
clf = clf.fit(bag, df['作者'])
predict = clf.predict(test_bag)
print("預測　　:", list(predict))
print("正確標籤:", list(test_df['作者']))
print("kNN 正確率: ", accuracy_score(test_df['作者'], predict) * 100, "%")

預測　　: [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 2, 0, 1, 1, 2, 2, 0, 0, 2, 0, 2]
正確標籤: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
kNN 正確率:  53.333333333333336 %


In [12]:
cm = confusion_matrix(test_df['作者'], predict)
pd.DataFrame(cm)

,0,1,2
0,7,3,0
1,4,5,1
2,4,2,4
